# layernorm

在大型语言模型，如Transformer类模型中使用的Layer Normalization技术，一般是对一个token在所有隐藏维度上进行的，而不是对整个sequence做的。Layer Normalization是一种归一化技术，它被应用在每个token的representation上，目的是为了减少训练过程中的内部协变量偏移问题（Internal Covariate Shift）。

具体来说，在Transformer模型的每个子层（如自注意力和前馈网络）的输出上，都会采用Layer Normalization。它通常被应用在multi-head attention的输出或者是position-wise feed-forward网络的输出之后。计算公式如下：

$$ \text{LN}(x_i) = \gamma \cdot \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta $$

其中，$ x_i $ 就是第i个token的隐藏层向量，$ \mu $ 是这个向量的平均值，$ \sigma^2 $ 是方差，$ \gamma $ 和 $ \beta $ 是可学习的参数（分别是scale和shift），而 $ \epsilon $ 是一个很小的数值，用于避免除以零。

Layer Normalization在每个时间步对每个token独立地计算其统计性质（平均值和标准差），使得在整个sequence中每个token的representation都进行了标准化。这与Batch Normalization不同，后者通常在batch的维度上进行归一化处理。

总结一下，Layer Normalization在大型语言模型中，针对的是单个token在其所有的feature dimensions上，而不是整个序列。

In [4]:
import torch
from torch import nn
# Image Example
N, C, H, W = 20, 5, 10, 10
input = torch.randn(N, C, H, W)
# Normalize over the last three dimensions (i.e. the channel and spatial dimensions)
# as shown in the image below
layer_norm = nn.LayerNorm([C, H, W])
output = layer_norm(input)

In [5]:
output.shape

torch.Size([20, 5, 10, 10])

在 PyTorch 的 `torch.nn.LayerNorm` 层中，参数 `normalized_shape` 指定了输入中要进行标准化的维度。当 `normalized_shape` 不是一维的时候，这意味着你想要对多维度进行层规范化操作（Layer Normalization）。

Layer Normalization是类似于Batch Normalization的一种标准化方法，但不同之处在于Layer Normalization是独立于批次大小的，而是对每个样本的指定维度进行标准化。这在很多场景中是有利的，因为它允许模型在训练和推理时拥有一致的行为，而不受批量大小的变化影响。

假设你有一个输入张量，其形状为 `[batch_size, num_features, height, width]`（例如一个小批量的图像，或者是卷积神经网络中一个卷积层的输出），并且你想要对最后两个维度（即 height 和 width）进行层规范化。在这种情况下，你会将 `normalized_shape` 设置为 `(height, width)`。这样，对于每个样本和每个特征，所有像素(pointwise)都会被考虑在内进行均值和标准差的计算，并将这些值用于标准化。

代码示例：

```python
import torch
from torch.nn import LayerNorm

# 假设 height 和 width 都是 10
normalized_shape = (10, 10)

# 创建 LayerNorm 层实例
layer_norm = LayerNorm(normalized_shape)

# 假设输入张量的形状为 [batch_size, num_features, height, width]
# 其中，batch_size=2, num_features=3, height=10, width=10
input_tensor = torch.rand(2, 3, 10, 10)

# 应用 LayerNorm
output = layer_norm(input_tensor)
```

在这个例子中，`LayerNorm` 会计算每个样本，每个特征映射（feature map）的均值和标准差，然后对该特征映射中的每个像素进行标准化。 计算时不会跨越不同的样本（batch中的不同元素）或者不同的特征映射（第二个维度）。 因此，这样的操作保证了每个特征映射独立于其它的样本和特征映射进行规范化，并以此增加模型的泛化能力。 

总的来说，`normalized_shape` 的多维度设置让你可以非常灵活地指定需要规范化的维度，这在处理多维数据时是非常有用的。

若你将一个形状为 `[batch_size, num_features, height, width]` 的输入张量 `input_tensor` 重塑（reshape）为 `[batch_size, num_features, height * width]`（在这个例子中为 `[2, 3, 100]`），接着使用层规范化（Layer Normalization）时指定 `normalized_shape` 为 `(height * width,)`（即 `(100,)`），你将会对最后一个维度进行标准化。

这种情况下，标准化操作在数学上是等效的，只是你现在将 `height` 和 `width` 的所有元素视为一个展平（flattened）的向量。标准化操作将计算这个向量的均值和标准差，然后使用这些值来标准化向量中的每个元素。

操作等效的要点在于，Layer Normalization 是针对每个样本中的特定维度展开运算的，无论你是把相应维度看作是独立的（如一张图片的长和宽），还是把它们合并为一维（如将图片像素展平）。只要在进行层规范化的维度上不发生跨越不同样本或通道（特征维度）的混合，操作就是等效的。

总结起来，当你将 `[2, 3, 10, 10]` 形状的张量重塑为 `[2, 3, 100]` 并且将 `normalized_shape` 设为 `(100,)` 来进行 `LayerNorm`，你在进行的是相同的数学运算，使得这两种操作在数学上是等价的。

在统计学中，一个估计量（estimator）是用来估计某个未知参数的统计规则或者算法。一个估计量被称为有偏的（biased）如果它的期望值不等于真正的参数值。换句话说，如果一个估计量系统地高估或低估了参数，那么它就是有偏的。

更正式地，如果参数的真实值是 $\theta$，估计量是 $\hat{\theta}$，那么 $\hat{\theta}$ 是有偏的，如果：

$$ E(\hat{\theta}) \neq \theta $$

这里 $E(\hat{\theta})$ 表示估计量 $\hat{\theta}$ 的期望值。

偏差（bias）的量是 $E(\hat{\theta}) - \theta$，表示估计量与真实参数值之间的差异。如果这个差异为零，即 $E(\hat{\theta}) = \theta$，那么 $\hat{\theta}$ 被称为无偏的（unbiased）。

一个经典的有偏估计量示例是在样本方差的计算中使用 $n$（样本大小）而不是 $n-1$ 作为分母。在这种情况下，所得的估计量会低估总体方差，因此它是有偏的。使用 $n-1$ 而不是 $n$ 可以得到一个无偏的样本方差估计。

重要的是要注意有偏估计量并不一定是一个坏的估计量。有时候一个有偏估计量可能具有其他的性质，例如较小的方差，这在某些情况下可能使其比无偏估计量更为可取。这种权衡是统计学中的一个重要概念，称为偏差-方差权衡（bias-variance tradeoff）。

在PyTorch中，使用函数`torch.var(input, unbiased=False)`可以计算输入张量的方差，其中参数`unbiased=False`指定了我们想要计算的是一个有偏的样本方差。当`unbiased`为`False`时，方差是除以`n`来计算的；而当它为`True`时，方差是除以`n-1`来计算的，其中`n`是样本数量。

当`unbiased=False`时，内部实现可以用标准Python代码大致模拟如下：

```python
import numpy as np

def biased_variance(data):
    n = len(data)
    mean = np.mean(data)  # 计算样本均值
    sum_squared_diffs = np.sum((data - mean) ** 2)  # 计算每个样本与均值之差的平方和
    variance = sum_squared_diffs / n  # 使用样本数量n来归一化
    return variance

# 假定有一个数据样本
data = np.array([1.0, 2.0, 3.0, 4.0, 5.0])

# 计算有偏方差
print(biased_variance(data))
```

在PyTorch中，计算方差的实现会被优化并使用GPU（如果可用的话），但基本概念与上述Python代码类似。通过设置参数`unbiased=True`，PyTorch会使用`n-1`而非`n`作为分母，这通常被称为Bessel's correction，用以获得无偏样本方差估计。

这里展示的代码是纯Python实现，使用了NumPy来模拟PyTorch的行为，因为实际的PyTorch实现会用C++编写，并利用其张量表示的特性来进行优化和并行处理。如果需要严格的实际PyTorch代码模拟，应当查阅PyTorch的源代码。

当我们使用样本数据来估计总体参数（比如总体方差）时，可以选择计算有偏估计或无偏估计。

有偏估计（Biased Estimator）:
有偏估计意味着估计器的期望值不等于要估计的总体参数。换句话说，有偏估计可能不会随着样本数量的增加而趋近于总体参数。样本方差的一个常见的有偏估计公式是：
$$ s^2 = \frac{1}{n} \sum_{i=1}^n (x_i - \bar{x})^2 $$
式中 $ s^2 $ 是样本方差，$ n $ 是样本大小，$ x_i $ 是单个的样本观测值，而 $ \bar{x} $ 是样本均值。使用 $ n $ 作为分母得到的是有偏估计的样本方差。

无偏估计（Unbiased Estimator）:
无偏估计的特点是估计器的期望值等于要估计的总体参数。对于样本方差来说，如果我们想得到总体方差的无偏估计，我们需要使用以下无偏估计公式：
$$ s^2 = \frac{1}{n-1} \sum_{i=1}^n (x_i - \bar{x})^2 $$
相较于有偏估计，这里我们使用 $ n-1 $ 作为分母，这是因为在估计总体方差时，使用 $ n-1 $ 可以补偿由于仅使用样本数据而来的信息损失，这种方法被称为Bessel修正。这样计算出的 $ s^2 $ 是无偏的，随着样本数量的增加，其期望值会趋近于真实的总体方差。

为什么 $ n-1 $ 会让样本方差成为无偏估计呢？原因在于使用样本均值 $ \bar{x} $ 作为总体均值的代理导致了自由度的减少。因此，分母使用 $ n-1 $ 能够提供更好的估计来补偿这种减少的自由度。这种分母为 $ n-1 $ 的样本方差也称为“样本方差的无偏估计”。

简而言之，无偏估计提供了一个更准确反映总体参数的样本估计，在进行统计分析和推断时，无偏估计是非常重要的。在实际中，人们常常使用无偏方差估计来确保推断的准确性。